# Homework 3 - data transformation & dimensionality reduction (deadline 26. 12. 2020, 23:59)

In short, the main task is to play with transformations and dimensionality reduction to obtain the best results for the linear regression model predicting house sale prices.
  
> The instructions are not given in detail: It is up to you to come up with ideas on how to fulfill the particular tasks as best you can!

## What are you supposed to do:

Your aim is to optimize the _RMSLE_ (see the note below) of the linear regression estimator (=our prediction model) of the observed sale prices.

### Instructions:

  1. Download the dataset from the course pages (data.csv, data_description.txt). It corresponds to [this Kaggle competition](https://www.kaggle.com/c/house-prices-advanced-regression-techniques). 
  1. Transform features appropriately and prepare new ones - focus on the increase in the performance of the model (possibly in combination with further steps). Split the dataset into a train and test part exactly as we did in the tutorials. Use the test part for evaluation of the influence of further steps.
  1. Try to find some suitable subset of features - first without the use of PCA.
  1. Use PCA (principal component analysis) to reduce the dimensionality. Discuss the influence of the number of principal components.
  1. Compare the results of previous steps on the test part of the dataset.
  
Give comments (!) on each step of your solution, with short explanations of your choices.

**If you do all this properly, you will obtain 16 points.** 


**Note**: _RMSLE_ is a Root-Mean-Squared-Error (RMSE) between the logarithm of the predicted value and the logarithm of the observed sale prices.


## Comments

  * Please follow the instructions from https://courses.fit.cvut.cz/MI-PDD/homeworks/index.html.
  * If the reviewing teacher is not satisfied, she can (!) give you another chance to rework your homework and to obtain more points. However, this is not a given, so do your best! :)
  * English is not compulsory.

In [1]:
# imports
import numpy as np
import pandas as pd
import math

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression, chi2
from scipy import stats
from sklearn.decomposition import PCA
np.seterr(divide='ignore', invalid='ignore')

RANDOM_SEED = 21

In [2]:
df = pd.read_csv('data.csv')

In [3]:
# first look on the dataset
df.info()
display(df.describe())
display(df.head(2))
display(df.tail(2))
df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125
1459,1460,20,RL,75.0,9937,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,147500


Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [4]:
def isNotInteger(value):
    if not value.is_integer() and not np.isnan(value):
        print(value)
    return value

In [5]:
df.LotFrontage.apply(isNotInteger);

In [6]:
df.Street.unique()

array(['Pave', 'Grvl'], dtype=object)

In [7]:
df.MasVnrArea.apply(isNotInteger);

In [8]:
df.GarageYrBlt.apply(isNotInteger);

**Prvotní poznatky při pohledu na data:**
* Dataset je velmi malý (1460 záznamů)
* Má 80 sloupců z nichž je většina vyplněná nenulovými hodnotami
* Mnoho sloupců je typu int64, ale stále převažuje typ 'object', který by nám mohl výpočty modelu dost komplikovat. Po prozkoumání významů těchto sloupců a jejich hodnot v datasetu jsem se rozhodl je všechny přetypovat na kategorialní typ, čímž zjednoduším práci s datasetem pro model. Níže v bodech nechávám krátké poznámky k jednotlivým sloupcům.
    * MSZoning -> přímo v dokumentaci tento sloupec může nabývat pouze 8 hodnot
    * LotFrontage -> sice je float, ale (viz buňka výše používající _isNotInteger_ metodu) - všechny hodnoty ve sloupci jsou buď celo číselné, nebo **Nan** --> změním tedy typ na int
    * Po zkontrolování hodnot v ostaních sloupcích jsem si potvrdil, že nabývají omezeného množství hodnot. Tyto hodnoty jsou vždy definované v přiloženém dokumentu (vždy je to do 10 hodnot až na výjimky). Všechny sloupce je tedy možné zkategorizovat.
    * MasVnrArea i GarageYrBlt také obsahuje pouze celočíselné hodnoty => převedu na int

In [9]:
# transform all object types into categories
df.loc[:, df.dtypes == 'object'] = df.select_dtypes(['object']).apply(lambda x: x.astype('category'))
# transform all floats into ints
df.loc[:, df.dtypes == 'float64'] = df.select_dtypes(['float64']).fillna(0)
df.loc[:, df.dtypes == 'float64'] = df.select_dtypes(['float64']).apply(lambda x: x.astype('int64'))
# df.info()

### Vytváření nových a úprava aktuálních sloupců
* Datasetu jsem se pokusil přidat pár nových sloupců, které jsou snad logické a mohly by predikcím pomoct.
    * nový sloupec _newHouse_ značící, zda je dům nový/nově zrekontruovaný
    * zaokrouhlení sloupců s rozlohami na desítky
    * přidání binárních sloupců, které signalizují zda dům má např. bazén, plot apod.
    * konverze všech vzniklých boolean sloupců na int
    * všechny hodnoty v predikovaném sloupci _SalePrice_ jsem zlogaritmoval (viz. doporučení v zadání, abysme měřili chybu na zlogaritmovaných hodnotách)

In [10]:
def roundNumber(val):
    if val%10 >= 5:
        return val+10-val%10
    else:
        return val-val%10

In [11]:
def changeBooleanToInt(val):
    if val:
        return 1
    return 0

In [12]:
# column new/old house condition - based on YearRemodAdd column
df['newHouse'] = df.YearRemodAdd > 2000

# round are values
for column in df.filter(regex='Area|SF', axis=1).columns:  # SF always represents area value
    df[column] = df[column].apply(roundNumber)
    df['Has' + column] = df[column] > 0

# remove all constant columns (if there are any)
colsToRemove = []
for column in df.columns:
    if df.dtypes[column].name != 'category' and df[column].min() == df[column].max():
        colsToRemove.append(column)
df.drop(columns=colsToRemove, inplace=True)

# change all boolean columns to int
boolCols = []
for column in df.columns:
    if df.dtypes[column].name == 'bool':
        boolCols.append(column)
for column in boolCols:
    df[column] = df[column].apply(changeBooleanToInt)
    df[column] = df[column].astype('uint8')

# logaritmize predicted value
df['SalePrice'] = np.log(np.array(df['SalePrice']))

* Vzhledem k tomu, že většina vytvořených kategorií nabývá pouze pár hodnot, tak je možné a i vhodné použít _one-hot encoding_ díky nemuž bude i dataset lépe zpracovatelný pro model lineární regrese, který budeme používat.

In [13]:
df_original = df.copy()
df = pd.get_dummies(df)
# display(df.head(3))

Změna všech ne-dummy hodnot na jeden datový typ, pro přehlednost a hlavně pro lepší práci v následujících krocích (normalizaci dat)

In [14]:
# change all int64 columns to float64 type for better consistency
df[df.select_dtypes(['float64', 'int64']).columns] = df[df.select_dtypes(['float64', 'int64']).columns].astype('float64')
print(df.dtypes.value_counts())

uint8      264
float64     38
dtype: int64


### Definice funkcí pro opakované vyhodnocení modelu lineární regrese

In [15]:
def linearRegression(X_train, X_test, Y_train, Y_test, printOut = True):
    # fit Linear Regression model
    lr = LinearRegression()
    lr.fit(X_train, Y_train) 
    
    # show RMSLE on train and test sets
    rmsle_train = np.sqrt(mean_squared_error(lr.predict(X_train), Y_train))
    rmsle_test = np.sqrt(mean_squared_error(lr.predict(X_test), Y_test))

    if printOut:
        print('RMSLE na trénovací množině dat:', round(rmsle_train, 4))
        print('RMSLE na testovací množině dat:', round(rmsle_test, 4))
    return rmsle_test

In [16]:
def splitData(data):
    x = data.drop(columns=['SalePrice'])
    return train_test_split(x, data['SalePrice'], test_size=0.25, random_state=RANDOM_SEED)

### Rozdělení dat na trénovací a testovací množinu

In [17]:
X_train, X_test, Y_train, Y_test = splitData(df)
linearRegression(X_train, X_test, Y_train, Y_test)

RMSLE na trénovací množině dat: 0.0868
RMSLE na testovací množině dat: 0.1489


0.14894552460951335

In [18]:
previousBestRMSE_train = 0.0868
previousBestRMSE_test = 0.1489

### Normalizace datasetu
Nejprve jsem zkusil data znormalizovat. Normalizoval jsem každou množinu zvlášt a normalizoval jsem pouze číselné ne-dummy hodnoty. Zkusil jsem aplikovat 2 modely: StandardScaler a MinMaxScaler

In [19]:
def normalization(model):
    cols = pd.DataFrame(X_train).select_dtypes(include=['float64']).columns
    model.fit(X_train[cols])

    X_train2 = X_train.copy()
    X_train2[cols] = model.transform(X_train[cols])
    X_test2 = X_test.copy()
    X_test2[cols] = model.transform(X_test[cols])
    linearRegression(X_train2, X_test2, Y_train, Y_test)

In [20]:
print(previousBestRMSE_train)
print(previousBestRMSE_test)
normalization(StandardScaler())

0.0868
0.1489
RMSLE na trénovací množině dat: 0.1071
RMSLE na testovací množině dat: 37454045054.181


In [21]:
print(previousBestRMSE_train)
print(previousBestRMSE_test)
normalization(MinMaxScaler())

0.0868
0.1489
RMSLE na trénovací množině dat: 0.0868
RMSLE na testovací množině dat: 7210894053.7365


StandardScaler ani MinMaxScaler nebyli schopni data vhodně zlepšit, abych se rozhodl je pro tento dataset použít. Musím dodat, že jsem s nimi měl velké problémy, než jsem je vůbec rozchodil (ač se jedná o velmi primitivní kód). Asi 2x jsem přepisoval celou přípravu datasetu, než se mi povedlo modely na data pustit bez chyby ... a nakonec jsem dostal tak ustřelené hodnoty RMSLE, že ani nemá smysl žádný z modelů aplikovat na data ... no radost :D

#### Nyní se pokusím zlepšit predikce modelu pomocí tzv. výběru hodnot (sloupců) na základě známých technik, které by mohli datasetu pomoct. Zkusím celkem 3:
* SelectKBest - knihovní funkce k určení sloupců, které mají největší a nejlepší vliv na predikovanou proměnnou. Zde se pokusím najít i nejvhodnější k.
* Variance Threshold, který odebere všechny sloupce, jejichž rozptyl hodnot přesahuje stanovený threshold. I zde se pokusím najít rozumnou hodnotu pro threshold
* T-test, který měří, jaký mají jednotlivé hodnoty vliv na predikovanou proměnnou. Pak se zvolí vhodná hodnota a všechny sloupce, které hodnotu vlivu mají nižsí se vyřadí

**SelectKBest**

In [22]:
minRMSLE = math.inf
bestK = 0
for k in range(15, 301, 15):
    kBest = SelectKBest(score_func=f_regression, k=k)  # I have also experimented with chi2 scoring function
    X_train_kbest = kBest.fit_transform(X_train, Y_train)
    X_test_kbest = kBest.transform(X_test)
    
    rmsle =  linearRegression(X_train_kbest, X_test_kbest, Y_train, Y_test, False)
    if minRMSLE > rmsle:
        bestK = k
        minRMSLE = rmsle

print('Previous best RMSLE value:', previousBestRMSE_test)
print('Best RMSLE value:', round(minRMSLE, 4), 'Best k found:', bestK)

Previous best RMSLE value: 0.1489
Best RMSLE value: 0.1372 Best k found: 210


Metoda _SelectKBest_ nám pro hodnotu k=210 byla schopna poměrně hezky zlepšit predikce našeho modelu.

Aplikuji tedy tyto změny na datové sady **X_train** a **X_test** a uložím hodnotu jako nejlepší naměřenou.

In [23]:
# apply kBest selection to data
previousBestRMSE_test = minRMSLE
kBest = SelectKBest(score_func=f_regression, k=bestK)
X_train = kBest.fit_transform(X_train, Y_train)
X_test = kBest.transform(X_test)

**Variance Threshold**

In [24]:
minRMSLE = math.inf
bestThreshold = 0
for threshold in np.arange(0, 1, 0.01):
    vt = VarianceThreshold(threshold=threshold)
    X_train_vt = vt.fit_transform(X_train)
    X_test_vt = vt.transform(X_test)
    
    rmsle =  linearRegression(X_train_vt, X_test_vt, Y_train, Y_test, False)
    if minRMSLE > rmsle:
        bestThreshold = threshold
        minRMSLE = rmsle

print('Previous best RMSLE value:', previousBestRMSE_test)
print('Best RMSLE value:', round(minRMSLE, 4), 'Best threshold found:', bestThreshold)

Previous best RMSLE value: 0.13716631268548354
Best RMSLE value: 0.1372 Best threshold found: 0.0


Metoda _VarianceThreshold_ již nepřinesla žádné zlepšení, rozhodl jsem se ji tedy neaplikovat.

**T-test**

T-test jsem se rozhodl pro jednoduchost rovnou aplikovat na původní daframe před aplikací _SelectKBest_ metody

In [25]:
ttest_pvals = pd.DataFrame(df).select_dtypes(
    include = ['uint8']
).columns.to_series().apply(
    lambda x: stats.ttest_ind(
        df.SalePrice[df[x] == 0],
        df.SalePrice[df[x] == 1],
        equal_var = False
    ).pvalue
)

# display largest and smallest p-values
display(ttest_pvals.nlargest(5))
display(ttest_pvals.nsmallest(2))

C:\Users\OndraS\anaconda3\envs\s\lib\site-packages\numpy\core\fromnumeric.py:3622: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\OndraS\anaconda3\envs\s\lib\site-packages\numpy\core\fromnumeric.py:3622: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\OndraS\anaconda3\envs\s\lib\site-packages\numpy\core\fromnumeric.py:3622: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\OndraS\anaconda3\envs\s\lib\site-packages\numpy\core\fromnumeric.py:3622: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\OndraS\anaconda3\envs\s\lib\site-packages\numpy\core\fromnumeric.py:3622: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\OndraS\anaconda3\envs\s\lib\site-packages\numpy\core\fromnumeric.py:3622: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\OndraS\anaconda3\envs\s\lib\site-packages\numpy\core\fromnumeric.py:3622: RuntimeWarning: Degrees of freedom <= 0 for sli

SaleType_ConLI      0.941270
BsmtFinType2_GLQ    0.932949
MiscFeature_Gar2    0.902273
LotConfig_Corner    0.877743
LotConfig_FR2       0.823163
dtype: float64

ExterQual_TA    1.235959e-120
ExterQual_Gd    1.124822e-108
dtype: float64

Warningy výše jsou nejpíše způsobeny tím, že se v některých případech metoda snaží vypočítat kovarianci z 1 prvku, což vyhodí tento warning

In [26]:
# remove all larger than 70 %
colsToRemove = list(ttest_pvals[ttest_pvals > 0.7].index)
colsToRemove

['LandContour_Lvl',
 'LotConfig_Corner',
 'LotConfig_FR2',
 'RoofStyle_Mansard',
 'RoofMatl_Tar&Grv',
 'ExterCond_Ex',
 'BsmtFinType2_GLQ',
 'GarageQual_Ex',
 'GarageCond_Gd',
 'MiscFeature_Gar2',
 'SaleType_ConLI']

In [27]:
df_ttest = df.drop(columns=colsToRemove)

In [28]:
X_train_ttest, X_test_ttest, Y_train_ttest, Y_test_ttest = splitData(df)
linearRegression(X_train_ttest, X_test_ttest, Y_train, Y_test);

RMSLE na trénovací množině dat: 0.0868
RMSLE na testovací množině dat: 0.1489


T-test nebyl schopen dosáhnout tak dobrých výsledků jako _SelectKBest_, proto ho na výsledný dataset neaplikuji.

## PCA
Nyní zkusím feature selection pomocí PCA. Nejprve provedu PCA na původním dataframu (před aplikování _selectKBest_ metody) a následně zkusím PCA i na již upravených datech a budu pozorvat, zda výběr sloupců dokáže ještě nějak zúžit a zlepšit kvalitu predikcí. Jeden z hlavních argumentů této metody je `n_components`, který určuje počet tzv. výsledných komponent, které metoda zachová. Vzhledem k tomu, že se jedná o parametr, který má na zkvalitnění dat velký vliv, pokusím se najít jeho nejvhodnější hodnotu.

**PCA nad původních df**

In [29]:
# just the X_train and X_test sets were edited with selectKBest - df was not modified
X_train_orig, X_test_orig, Y_train_orig, Y_test_orig = splitData(df)

In [30]:
minRMSLE = math.inf
bestN = 0
for n in range(1, 300, 4):
    pca = PCA(n_components=n, random_state=RANDOM_SEED)
    X_train_pca = pca.fit_transform(X_train_orig, Y_train)
    X_test_pca = pca.transform(X_test_orig)
    
    rmsle =  linearRegression(X_train_pca, X_test_pca, Y_train, Y_test, False)
    if minRMSLE > rmsle:
        bestN = n
        minRMSLE = rmsle
print('Previous best RMSLE value:', previousBestRMSE_test)
print('Best RMSLE value:', round(minRMSLE, 4), 'Best n:', bestN)

Previous best RMSLE value: 0.13716631268548354
Best RMSLE value: 0.1395 Best n: 153


**PCA na již upravených datech**

In [31]:
minRMSLE = math.inf
bestN = 0
for n in range(1, 210, 4):
    pca = PCA(n_components=n, random_state=RANDOM_SEED)
    X_train_pca = pca.fit_transform(X_train, Y_train)
    X_test_pca = pca.transform(X_test)
    
    rmsle =  linearRegression(X_train_pca, X_test_pca, Y_train, Y_test, False)
    if minRMSLE > rmsle:
        bestN = n
        minRMSLE = rmsle
print('Previous best RMSLE value:', previousBestRMSE_test)
print('Best RMSLE value:', round(minRMSLE, 4), 'Best n:', bestN)

Previous best RMSLE value: 0.13716631268548354
Best RMSLE value: 0.1384 Best n: 189


Překvapivě PCA nepředčila zlepšení metody _SelectKBest_ a ani ho již nadále nedokázala zlepšit.

## Závěr

Povedlo se mi zpracovat dataset a připravit jeho sloupce pro následnou práci s modelem lineární regrese. Dále se mi povedlo vytvořit pár nových (snad smysluplných) sloupců. Následně jsem se pokusil některé sloupce znormalizovat, ale to bohužel bez úspěchu (ještě jsem přitom narazil na velké problémy, z kterých je mi doteď smutno). Dále jsem zkusil manuální feature selection vhodných sloupců, abych zjednodušil data a zároveň zlepšil predikce. U jedné metody bylo zlepšení dobré a to dokonce natolik, že ani metodě PCA, kterou jsem také na data zkusil aplikovat, se nepodařilo je předčít.